# Actuarial Standards of Practice (ASOP) Q&A Machine using Retrieval Augmented Generation (RAG)
This project aims to create a Retrieval-Augmented Generation (RAG) process for actuaries to ask questions on a set of Actuarial Standards of Practice (ASOP) documents. The RAG process utilizes the power of the Large Language Model (LLM) to provide answers to questions on ASOPs.

However, RAG is not without challenges, i.e., hallucination and inaccuracy. This code allows verifiability by providing the context it used to arrive at those answers. This process enables actuaries to validate the information provided by the LLM, empowering them to make informed decisions. By combining the capabilities of LLM with verifiability, this code offers actuaries a robust tool to leverage LLM technology effectively and extract maximum value.

The current example uses either OpenAI's GPT 3.5 turbo or a local LLM. Using a local LLM can address potential data privacy or security concerns.

View license or further information about the local models used:
- Solar 10.7B Instruct: [cc-by-nc-4.0](https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0) (non-commercial use)
- Mistral 7B Instruct: [Apache License 2.0](https://ollama.com/library/mistral/blobs/sha256:43070e2d4e532684de521b885f385d0841030efa2b1a20bafb76133a5e1379c1)
- [GPT4All embedding model](https://python.langchain.com/docs/integrations/text_embedding/gpt4all)

# 1. Initial Setup
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.


In [4]:
# Initial set up
from dotenv import load_dotenv
import os

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  # This loads the variables from .env (not part of repo)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["LANGCHAIN_TRACING_V2"] = "true" # use when you want to debug or monitor the performance of your langchain applications
#os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # use when accessing cloud-based language models or services that langchain integrates with

# Import the necessary modules
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown, Latex
import glob
import chromadb

In [5]:
# Change to False if using local models instead of OpenAI models
use_OpenAI = True

if use_OpenAI:
    embeddings_model = OpenAIEmbeddings()
    db_directory = "../data/chroma_db1"
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", 
                     temperature=0) # context window size 16k for GPT 3.5 Turbo

else: #Open source models used here are for illustration and educational purposes
    embeddings_model = GPT4AllEmbeddings()
    db_directory = "../data/chroma_db2"
    # define a local large language model for the augmented generation
    # Ollama is one way to easily run inference
    #llm = Ollama(model="solar:10.7b-instruct-v1-q5_K_M")
    llm = Ollama(model="mistral:instruct")
    

# 2. Load PDF Files and Convert to a Vector DB
1. Create a function to load and extract text from PDF files in a specified folder. It defines a function called `load_pdfs_from_folder()` that takes a folder path as input and returns a list of extracted text documents from the PDF files in that folder.

2. In the example, the folder path `../data/ASOP` is used, but you can modify it to point to your desired folder.

3. By calling the `load_pdfs_from_folder()` function with the folder path, the code loads the PDF files, extracts the text using the PyPDFLoader, and stores the extracted text documents in the `docs` list.

4. After loading and extracting the text, a `RecursiveCharacterTextSplitter` object is created with specific parameters for chunking the documents. The `split_documents()` method is then used to split the documents into smaller chunks based on the specified parameters.

5. Finally, a Chroma vectorstore is created from the document splits. The vectorstore uses the defined embedding model for embedding the chunks and is saved to the predefined directory.

In [6]:
# Run only when the DB directory is empty
if not os.path.exists(db_directory) or not os.listdir(db_directory):
    # Define a function to load and extract text from PDFs in a folder
    def load_pdfs_from_folder(folder_path):
        # Get a list of PDF files in the specified folder
        pdf_files = glob.glob(f"{folder_path}/*.pdf")
        docs = []
        for pdf_file in pdf_files:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file) 
            # Extract the text from the PDF and add it to the docs list
            docs.extend(loader.load())
        return docs
    
    # Example folder path
    folder_path = '../data/ASOP'
    
    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        length_function=len,)
    
    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)
    
    # Create a Chroma vector database from the document splits, using OpenAIEmbeddings for embedding
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=embeddings_model, 
                                        persist_directory=db_directory)

# 3. Retrieve from the Vector DB 

In [7]:
# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory)

In [9]:
## Retrieve and RAG chain

# Create a retriever using the vector database as the search source
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.35}) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# 4. Generate Q&A Function

In [10]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

# Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [11]:
generate_output()

What is your question on ASOP?:  Explain ASOP No 14


### Question
Explain ASOP No 14

### Answer
ASOP No. 14 was reviewed as part of a project to look at all cash flow testing standards of practice. Relevant portions of ASOP No. 14 were incorporated within the 2001 revisions of ASOP No. 7 and ASOP No. 22. The ASB voted to repeal ASOP No. 14 in September 2001.

### Context
- **Source 1**: ../data/ASOP/asop007_128.pdf, page 5:

virelated to cash flow testing. Finally, the ASB has adopted a new format for standards, and this   
standard has been rewritten to conform to that new format.  In addition to ASOP No. 7, as part of the project  to look at all cash flow testing standards of   
practice, ASOP No. 14 and ASOP No. 22 were al so reviewed. Relevant portions of ASOP No.   
14 were incorporated within the 2001 revisions of ASOP No. 7 and ASOP No. 22.   At its September 2001 meeting, the ASB voted to adopt the revised ASOP No. 7 and ASOP No. 22 and to repeal ASOP No. 14. In April 2002, the ASB voted to defer the effective date of ASOP No. 7 to July 15, 2002 while it reviewed concerns raised by the Academy’s Casualty Practice Council regarding the standard’s applicability to property/casualty practice. At its June 2002 meeting, the ASB amended the scope to conform to generally accepted casualty actuarial practice. Please see appendix 3 for further information.   Exposure Draft
- **Source 2**: ../data/ASOP/asop004_173.pdf, page 31:

are found in the current version of ASOP No. 4. The reviewers believe the reference to Precept 8   
remains appropriate. The reviewers do not believe that the proposed change significantly improves   
the language included in the current version of ASOP No. 4, and made no change.
- **Source 3**: ../data/ASOP/asop036_153.pdf, page 20:

commentator.   
Comment   
   
   
Response Two commentators noted the references to ASOP No. 41 correspond to an exposure   
draft rather than the standard in place.   
   
This final version refers to the final version of ASOP No. 41 effective April 1, 2011.
- **Source 4**: ../data/ASOP/asop044_160.pdf, page 3:

ASOP No. 44—September 2009    
   
   
iv   
  September 2009   
 TO: Members of Actuarial Organizations Gove rned by the Standards of Practice of the   
Actuarial Standards Board and Other Persons  Interested in the Selection and Use   
of Asset Valuation Methods for Pension Valuations   
 FROM:  Actuarial Standards Board (ASB)   
 SUBJ:  Actuarial Standard of Practice (ASOP) No. 44   
  This document contains the clar ified version of ASOP No. 44, Selection and Use of Asset   
Valuation Methods for Pension Valuations .    
   
Background  
   
 ASOP No. 44 was issued as a new standard in September 2007, with an effective date of March   
15, 2008. The ASB recently became aware of a need for clarification of the wording in section   
3.4.1, Bias. The intent of the section is to require di sclosure of the existence of systematic bias in   
the asset valuation method only when such bias  is, in the actuary’s professional judgment,
- **Source 5**: ../data/ASOP/asop016_104.pdf, page 6:

while the other two offered comments and sugge stions. Two commentators either oppos ed or offered an alternative to repealing ASOP No. 16. The ASB, Health Committee, and Task Force to Revise ASOP No. 16 carefully considered all comments received. Summarized below are the significant issues and questions contained in the comments and respons es to each.  GENERAL COMMENTS Comment    Response One commentator suggested that ASOP No. 16 be retained with all text being deleted except the references to the ASOPs that are appropriate to HMOs and other managed-care health plans.  This repeal document lists the ASOPs that provide guidance for HMOs and other managed-care health plans. In addition, appendix 1 has been added, which shows sections 2 and 5 of ASOP No. 16, and whether guidance is provided in other ASOPs, by NAIC instructions, or by the Code of Professional Conduct, or the material is considered educational and is not appropriate for inclusion in an ASOP. Comment      Response One
- **Source 6**: ../data/ASOP/asop026_144.pdf, page 8:

c. the disclosure in ASOP  No. 41, section 4.4, if, in the actuary’s professional   
judgment, the actuary has othe rwise deviated materially from the guidance of this   
ASOP.


# 5. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://python.langchain.com/docs/integrations/vectorstores/chroma
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/
- https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html

In [12]:
vectorstore._collection.name # name of the collection

'langchain'

In [13]:
vectorstore._collection.count() # Number of collections

4754

In [14]:
vectorstore._collection.peek(1) # See what the first component of the vectorstore looks like

{'ids': ['284046d6-ce8e-11ee-9579-0e2e3f1e4b98'],
 'embeddings': [[-0.0061185057274997234,
   0.0022151446901261806,
   0.018539685755968094,
   -0.029903599992394447,
   0.0011936883674934506,
   0.025374405086040497,
   -0.00987009797245264,
   0.0045974054373800755,
   -0.02012217603623867,
   -0.017175471410155296,
   -0.0015517949359491467,
   0.021445464342832565,
   0.011391198262572289,
   0.01742102950811386,
   0.005126039031893015,
   0.024965140968561172,
   0.012509854510426521,
   -0.02508791908621788,
   0.017666589468717575,
   0.017039049416780472,
   -0.027966413646936417,
   0.016902627423405647,
   -0.04057176411151886,
   0.004498499911278486,
   -0.00495210150256753,
   0.01351937372237444,
   0.014324260875582695,
   -0.03181350231170654,
   0.019494637846946716,
   0.02104984223842621,
   0.016629785299301147,
   -0.012557602487504482,
   -0.025046993046998978,
   -0.022236710414290428,
   -0.007530468516051769,
   -0.011875494383275509,
   -0.006848360877484083